## Installing Sentence Transformer and other models/frameworks

In [ ]:
!pip install sentence_transformers
!pip install transfomers

#Getting the glove model from stanfordnlp
!wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.42B.300d.zip

# Unzipping ther obtained zip file of glove model
!unzip /content/glove.42B.300d.zip

# Kindly add all your installations and versions if any in this cell.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.6 MB/s 
     |████████████████████████████████| 5.5 MB 12.4 MB/s 
     |████████████████████████████████| 1.3 MB 33.6 MB/s 
     |████████████████████████████████| 182 kB 46.5 MB/s 
     |████████████████████████████████| 7.6 MB 4.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9ac93931cf8b6e14cfea92ee140081db2ecd8bcde7147e5bc5b05f74e69917df
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement transfomers (from versions: none)
ERROR: No matching distribution found for transfomers
--2022-11-30 15:23:11--  https://hu

## Importing necessary libraries. 
In the final version all imports should be stricly enlisted here.

In [ ]:
import pandas as pd
import numpy as np
import spacy
from scipy import stats
from sklearn import linear_model

from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

import torch 
from torch.utils.data import DataLoader

import zipfile
from zipfile import ZipFile

from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from google.colab import drive
drive.mount('/content/drive/')

from numpy.random import rand
from numpy.random import seed
from scipy.stats import spearmanr

from sentence_transformers import SentenceTransformer, models, losses

import re
import nltk
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from sentence_transformers import InputExample

from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download("punkt")

Mounted at /content/drive/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Load dataset: 7 marks
1 Download and unzip the dataset from this link http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz  **1 mark**

2 Complete the code in `read_sts_csv()`. **4.5 marks**

3 Create 3 dataframes one each for train, test and val and print their final shapes. **1.5 marks**

In [ ]:
!wget http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
!tar -xvf  '/content/Stsbenchmark.tar.gz'

--2022-11-30 15:31:52--  http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.es (ixa2.si.ehu.es)... 158.227.106.100
Connecting to ixa2.si.ehu.es (ixa2.si.ehu.es)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz [following]
--2022-11-30 15:31:53--  http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.eus (ixa2.si.ehu.eus)... 158.227.106.100
Connecting to ixa2.si.ehu.eus (ixa2.si.ehu.eus)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409630 (400K) [application/x-gzip]
Saving to: ‘Stsbenchmark.tar.gz’

Stsbenchmark.tar.gz 100%[===================>] 400.03K   447KB/s    in 0.9s    

2022-11-30 15:31:54 (447 KB/s) - ‘Stsbenchmark.tar.gz’ saved [409630/409630]

stsbenchmark/
stsbenchmark/readme.txt
stsbenchmark/sts-test.csv
stsbenchmark/correlation.pl
stsbenchmark/LICENSE.txt
stsb

The read_sts_csv method reads the Semantic Text Similarity Dataset (STS) and returns the train,dev, and test dataframes. This function first opens the file and then read every line from it. Every line read is split with tab as the deliminiter and the components of it are stripped(removing extra spaces at the beginning or ending in it). These processed components are stored in the list which stored in another list. The latter list is then converted into data frame with the given columns and thus this dataframe is returned and thus data is loaded

In [ ]:
def read_sts_csv(dataset_type="train", columns=['source', 'type', 'year', 'id', 'score', 'sent_a', 'sent_b']):
  path = "/content/stsbenchmark/" + "sts-"+ dataset_type + ".csv"
  with open(path, "r", encoding="utf-8") as mfile:
      sentencePairs = []
      for Line in mfile:
          Line = Line.strip().split("\t")
          sentencePairs.append([Line[0].strip(), Line[1].strip(), Line[2].strip(), Line[3].strip(), float(Line[4]), Line[5].strip(), Line[6].strip(),])
      sentenceDataFrame = pd.DataFrame(sentencePairs, columns=columns,)
  return sentenceDataFrame

df_train = read_sts_csv("train")
print("Train Dataframe shape:",df_train.shape)
df_dev = read_sts_csv("dev")
print("Dev Dataframe shape:",df_dev.shape)
df_test = read_sts_csv("test")
print("Test Dataframe shape:",df_test.shape)

Train Dataframe shape: (5749, 7)
Dev Dataframe shape: (1500, 7)
Test Dataframe shape: (1379, 7)


The load_glove_model method returns a dictionary having the key as word and the value as the word's embedding

In [ ]:
def load_glove_model(File):
    print("Loading Glove Model")
    glove_model = {}
    with open(File,'r') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    print(f"{len(glove_model)} words loaded!")
    return glove_model

## Hyperparameters: 5 Marks
Update this cell with you choosen parameters except, NUM_EPOCHS

In [ ]:
NON_CONEXTUAL_MODEL_TYPE = load_glove_model("/content/glove.42B.300d.txt")
CONEXTUAL_MODEL_TYPE = SentenceTransformer('all-MiniLM-L6-v2')
HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL = models.Transformer('sentence-transformers/all-distilroberta-v1') # USE THE HUGGAING FACE VERSION OF SENTENCE_TRANSFORMER_TYPE
INPUT_PATH = '/content/stsbenchmark/'
BATCH_SIZE = 16
OUT_DIM_DENSE = 150
NUM_EPOCHS = 1 ## THIS IS FIXED DO NOT CHANGE

# You are free to add your own hyperparameters as well.

Loading Glove Model
1917495 words loaded!


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

The text_preprocessing_only_tokenization function accepts the dataframe as input and returns the list of sentences with and without tokenizing

In [ ]:
def text_preprocessing_only_tokenization(dataFrame):
  sentences1 = list(dataFrame['sent_a'])
  sentences2 = list(dataFrame['sent_b'])
  sentences1Tokens = []
  sentences2Tokens = []
  for i in range(len(sentences1)):
    sentences1Tokens.append([token for token in nltk.word_tokenize(sentences1[i])])
    sentences2Tokens.append([token for token in nltk.word_tokenize(sentences2[i])])
  return sentences1, sentences2, sentences1Tokens, sentences2Tokens

The text_preprocessing function accepts the dataframe as input and returns the list of sentences after and before applying pre-processing techniques like lower-casing,word tokenization,extra whitespace removal,non word character removal,etc

In [ ]:
def text_preprocessing(dataFrame):
  sentences1 = list(dataFrame['sent_a'])
  sentences2 = list(dataFrame['sent_b'])
  sentences1Tokens = []
  sentences2Tokens = []
  for i in range(len(sentences1)):
    sentences1[i] = sentences1[i].lower()
    sentences1[i] = re.sub('[^a-zA-Z]', ' ', sentences1[i])
    sentences1[i] = re.sub(r'\s+', ' ', sentences1[i])
    sentences1Tokens.append([token for token in nltk.word_tokenize(sentences1[i]) if token not in stopwords.words("english")])
    sentences2[i] = sentences2[i].lower()
    sentences2[i] = re.sub('[^a-zA-Z]', ' ', sentences2[i])
    sentences2[i] = re.sub(r'\s+', ' ', sentences2[i])
    sentences2Tokens.append([token for token in nltk.word_tokenize(sentences2[i]) if token not in stopwords.words("english")])

  return sentences1, sentences2, sentences1Tokens, sentences2Tokens

## CONFIGURATION 1: Non-contextual Embeddings + ML Regression: 8 marks
1 Load the non-contextual embedding model in variable `non_cont_model1`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model1()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model1`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model1`. **1.5 mark**



OUTPUT EMBEDDINGS OF SIZE: 300

The get_average method takes the sentence_tokens and the model as inputs and returns the embedding vector for the sentence.It takes the average of individual word embeddings to generate the embedding for the entire sentence.

In [ ]:
def get_average(sentence_tokens, model):
  sentence_tokens = [word for word in sentence_tokens if word in model.keys()]
  if sentence_tokens:
    sentence_embedding = np.zeros((len(sentence_tokens), 300), dtype=np.float32)
    for idx, word in enumerate(sentence_tokens):
      sentence_embedding[idx] = model[word]
  
    return np.mean(sentence_embedding, axis=0)
  else:
    return np.zeros(300)

Helper function for the above computation

In [ ]:
def sentence_embedding(list_of_tokens, model):
  sentence_embeddings = np.array([get_average(sentence_tokens, model) for sentence_tokens in list_of_tokens])  
  return sentence_embeddings

The cal_spearmanr_correlation method returns the Spearmans correlation coefficient for the given two data samples

In [ ]:
def cal_spearmanr_correlation(data1, data2):
  seed(1)
  coefficient, pValue = spearmanr(data1, data2)
  print('Spearmans correlation coefficient: %.3f' % coefficient)
  alphaParameter = 0.05
  if pValue > alphaParameter:
    print('Samples are uncorrelated (fail to reject H0) p=%.3f' % pValue)
  else:
    print('Samples are correlated (reject H0) p=%.3f' % pValue)

In [ ]:
def get_feature_model1(data_frame):
  """
  Input a data frame and return the embedding vectors for the each sentence column using non_cont_model1,
  Return 2 matrices each of shape (#_samples, #size_of_word_emb).
  """
  sentences1, sentences2, sentences1Tokens, sentences2Tokens = text_preprocessing_only_tokenization(data_frame)
  non_cont_model1 = NON_CONEXTUAL_MODEL_TYPE
  sentences1Embeddings = sentence_embedding(sentences1Tokens, non_cont_model1)
  sentences2Embeddings = sentence_embedding(sentences2Tokens, non_cont_model1)
  return np.asmatrix(sentences1Embeddings), np.asmatrix(sentences2Embeddings)


# feature_1_<dataset_type>, feature_2_<dataset_type> = get_feature_model1(data_frame)
feature_1_train, feature_2_train = get_feature_model1(df_train)
feature_1_dev, feature_2_dev = get_feature_model1(df_dev)
feature_1_test, feature_2_test = get_feature_model1(df_test)

# X_<dataset_type>, Y_<dataset_type> = 
X_train, Y_train = np.concatenate((feature_1_train, feature_2_train), axis=1), list(df_train['score'])
X_dev, Y_dev = np.concatenate((feature_1_dev, feature_2_dev), axis=1), list(df_dev['score'])
X_test, Y_test = np.concatenate((feature_1_test, feature_2_test), axis=1), list(df_test['score'])


In [ ]:
feature_1_train.shape

(5749, 300)

In [ ]:
feature_2_train.shape

(5749, 300)

In [ ]:
feature_1_test.shape

(1379, 300)

In [ ]:
feature_1_dev.shape

(1500, 300)

We apply scikit-learns's Linear regression method to model the relation between the input sentences and the corresponding similarity score.

In [ ]:
# Initiate a regression model and train it.
model1 = LinearRegression()
model1.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


LinearRegression()

In [ ]:
# Print spearmanr correlation on the predicted output of the dev set.
prediction_dev_set = model1.predict(X_dev)
cal_spearmanr_correlation(prediction_dev_set, Y_dev)

Spearmans correlation coefficient: 0.116
Samples are correlated (reject H0) p=0.000


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
# Print spearmanr correlation on the predicted output of the test set.
prediction_test_set = model1.predict(X_test)
cal_spearmanr_correlation(prediction_test_set, Y_test)

Spearmans correlation coefficient: 0.244
Samples are correlated (reject H0) p=0.000


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


## CONFIGURATION 2: Contextual Embeddings + ML Regression: 7 marks
1 Load the contextual embedding model in variable `non_cont_model2`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model2()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model2`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model2`. **1.5 mark**

Useful references: https://www.sbert.net/docs/usage/semantic_textual_similarity.html

OUTPUT EMBEDDINGS OF SIZE: 384

In [ ]:
def get_feature_model2(data_frame):
  """
  Input a data frame and return the embedding vectors for the each sentence column using model2,
  Return 2 matrices each of shape (#_samples, #size_of_word_emb).
  """
  sentences1, sentences2, sentences1Tokens, sentences2Tokens = text_preprocessing(data_frame)
  cont_model2 = CONEXTUAL_MODEL_TYPE
  sentences1Embeddings = cont_model2.encode(sentences1)
  sentences2Embeddings = cont_model2.encode(sentences2)
  return np.asmatrix(sentences1Embeddings), np.asmatrix(sentences2Embeddings)

# feature_1_<dataset_type>, feature_2_<dataset_type> = get_feature_model2(data_frame)
feature_1_train, feature_2_train = get_feature_model2(df_train)
feature_1_dev, feature_2_dev = get_feature_model2(df_dev)
feature_1_test, feature_2_test = get_feature_model2(df_test)

# X_<dataset_type>, Y_<dataset_type> = 
X_train, Y_train = np.concatenate((feature_1_train, feature_2_train), axis=1), list(df_train['score'])
X_dev, Y_dev = np.concatenate((feature_1_dev, feature_2_dev), axis=1), list(df_dev['score'])
X_test, Y_test = np.concatenate((feature_1_test, feature_2_test), axis=1), list(df_test['score'])

In [ ]:
feature_1_train.shape

(5749, 384)

In [ ]:
feature_2_train.shape

(5749, 384)

In [ ]:
feature_1_test.shape

(1379, 384)

In [ ]:
feature_1_dev.shape

(1500, 384)

We apply scikit-learns's Linear regression method to model the relation between the input sentences and the corresponding similarity score.

In [ ]:
# Initiate a regression model and train it.
model2 = LinearRegression()
model2.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


LinearRegression()

In [ ]:
# Print spearmanr correlation on the predicted output of the dev set.
prediction_dev_Set = model2.predict(X_dev)
cal_spearmanr_correlation(prediction_dev_Set, Y_dev)

Spearmans correlation coefficient: 0.081
Samples are correlated (reject H0) p=0.002


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
# Print spearmanr correlation on the predicted output of the test set.
prediction_test_Set = model2.predict(X_test)
cal_spearmanr_correlation(prediction_test_Set, Y_test)

Spearmans correlation coefficient: 0.251
Samples are correlated (reject H0) p=0.000


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


## CONFIGURATION 3: Fine-Tune a Contextual Embeddings Model: 18 marks
1 Prepare data samples to be for the DL model to consume. Add the code in the `form_data()`. **4 marks**

3 Create the data loader, one each for train/dev/test data_input sample set obtained from `form_input_example()`. **1.5 marks**

4 Initiate `model3` consisting of **atleast** the following 3 components - `base_LM`, a `pooling_layer` and a `dense_layer`. Use appropriate activation function in dense. **Atleast** one layer of `base_LM` should be set to trainable. **5 marks**

6 Initiate the `loss`. **0.5 marks**

7 Fit the `model3`. Use `NUM_EPOCHS = 2`. **MAX_NUM_EPOCHS allowed will be 3**. **2 marks** 

8 Complete the `get_model_predicts()` to obtain predicted scores for input sentence pairs. **3.5 marks** 

9 Print the correlation scores on the dev and test set predictions. **1.5 mark**

Useful References: https://huggingface.co/blog/how-to-train-sentence-transformers 

SIZE OF OUTPUT EMBEDDINGS BY THE

Base LM: 768

Pooling layer: 768

dense Layer: 150

THIS FUNCTION MAKES LAST LAYER OF THE BASE MODEL TRAINABLE BY SETTING requires_grad=False for all N-1th layers and requires_grad=True for the Nth Layer, where N are the number of layers in the model

In [ ]:
def change_param_req_grad(model, makeChange):
    if makeChange:
      numberOfLayers = sum(1 for _ in model.parameters())
      count = 1
      param_generator=model.parameters()
      while True:
        if(count > numberOfLayers-1):
          break
        else:
          param = next(param_generator)
          param.requires_grad = False
        count += 1

In [ ]:
def form_data(data_frame):
  """
  Input a data frame and return the dataloder.
  """
  samples = []
  n_samples = data_frame.shape[0]

  for i in range(n_samples):
    samples.append(InputExample(texts=[data_frame.loc[i,'sent_a'], data_frame.loc[i,'sent_b']], label=float(data_frame.loc[i,'score'])))
  dataloader = DataLoader(samples, shuffle=True, batch_size=BATCH_SIZE)
  return dataloader

def get_model_predicts(data_type, trained_model):
  """
  Input the dataset list and return a list of cosine similarity scores. Use the fitted final_trainable_model for obtaining encodings.
  """
  score_samples = []
  for i in range(len(data_type)):
    sample = data_type[i]
    score_samples.append(cosine_similarity(np.array([trained_model.encode(sample[0])]), np.array([trained_model.encode(sample[1])])))
  return score_samples


In [ ]:

# dataloader_<dataset_type> = form_data(data_frame)
dataloader_train = form_data(df_train)
dataloader_dev = form_data(df_dev)
dataloader_test = form_data(df_test)

# base_model = 
base_model = HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL
change_param_req_grad(base_model, True)

# layer_ppoling = 
layer_ppoling = models.Pooling(base_model.get_word_embedding_dimension())

# layer_dense = 
layer_dense = models.Dense(in_features=layer_ppoling.get_sentence_embedding_dimension(), out_features=OUT_DIM_DENSE, activation_function=nn.Tanh())

# model3 = 
model3 = SentenceTransformer(modules=[base_model, layer_ppoling, layer_dense])

# loss =
loss = losses.CosineSimilarityLoss(model3)

# Fit the model3.
model3.fit(train_objectives=[(dataloader_train, loss)], epochs=NUM_EPOCHS, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Print spearman correlation on the predicted output of the dev and test sets.

In [ ]:
dev_samples = [[df_dev.loc[i, 'sent_a'], df_dev.loc[i, 'sent_b']] for i in range(df_dev.shape[0])]
predicted_score_dev = get_model_predicts(dev_samples, model3)
test_samples = [[df_test.loc[i, 'sent_a'], df_test.loc[i, 'sent_b']] for i in range(df_test.shape[0])]
predicted_score_test = get_model_predicts(test_samples, model3)

In [ ]:
Y_dev = list(df_dev['score'])
Y_dev = [element/5 for element in Y_dev]
Y_test = list(df_test['score'])
Y_test = [element/5 for element in Y_test]

In [ ]:
cal_spearmanr_correlation(np.array(predicted_score_dev)[:, 0], np.array(Y_dev))

Spearmans correlation coefficient: 0.868
Samples are correlated (reject H0) p=0.000


In [ ]:
cal_spearmanr_correlation(np.array(predicted_score_test)[:, 0], np.array(Y_test))

Spearmans correlation coefficient: 0.807
Samples are correlated (reject H0) p=0.000


CONTRIBUTIONS:

SARTHAK MAINI(2020576): Q1 AND Q2

ABHIT RANA(2020421): Q3